In [130]:
import pandas as pd
import re
import plotly.express as px
import streamlit as st
from PIL import Image
import folium
from streamlit_folium import folium_static

In [177]:
# Lendo do Arquivo
df = pd.read_csv( 'zomato.csv' )
df['Cuisines']

array(['Italian', 'Italian', 'European, Asian', ..., 'Restaurant Cafe',
       'Home-made, Izgara', 'Restaurant Cafe, Kebab, Turkish Pizza'],
      dtype=object)

## Códigos fornecidos pelo Pleno

### Preenchimento do nome dos paises 

In [132]:
# Criação de lista para substituir o código dos países por nomes
COUNTRIES = {
1: "India",
14: "Australia",
30: "Brazil",
37: "Canada",
94: "Indonesia",
148: "New Zeland",
162: "Philippines",
166: "Qatar",
184: "Singapure",
189: "South Africa",
191: "Sri Lanka",
208: "Turkey",
214: "United Arab Emirates",
215: "England",
216: "United States of America",
}
df.aux = df
# Substituir todos os países de códigos para nomes
df.aux['Country Code'] = df['Country Code'].replace(COUNTRIES)


C:\Users\thiag\AppData\Local\Temp\ipykernel_7984\1799797935.py:19: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  df.aux = df


### Criação do Tipo de Categoria de comida

In [133]:
def create_price_type(price_range):
    if price_range == 1:
        return "cheap"
    elif price_range == 2:
        return "normal"
    elif price_range == 3:
        return "expensive"
    else:
        return "gourmet"


### Criação do Nome das Cores

In [134]:
COLORS = {
    "3F7E00": "darkgreen",
    "5BA829": "green",
    "9ACD32": "lightgreen",
    "CDD614": "orange",
    "FFBA00": "red",
    "CBCBC8": "darkred",
    "FF7800": "darkred",
}
def color_name(color_code):
    return COLORS[color_code]


### Renomear as Colunas do DataFrame

In [135]:
def rename_columns(dataframe):
    df = dataframe.copy()
    title = lambda x: inflection.titleize(x)
    snakecase = lambda x: inflection.underscore(x)
    spaces = lambda x: x.replace(" ", "")
    cols_old = list(df.columns)
    cols_old = list(map(title, cols_old))
    cols_old = list(map(spaces, cols_old))
    cols_new = list(map(snakecase, cols_old))
    df.columns = cols_new
    return df

### Categorizar os restaurantes por um tipo de culinária

In [136]:
df["Cuisines"] = df.loc[:, "Cuisines"].apply(lambda x: str(x).split(",")[0])


## Desafio - Geral

#### 1. Quantos restaurantes únicos estão registrados?


In [137]:
df['Restaurant ID'].count()

7527

#### 2. Quantos países únicos estão registrados?


In [138]:
df['Country Code'].value_counts().count()

15

#### 3. Quantas cidades únicas estão registradas?


In [139]:
df['City'].value_counts().count()

125

#### 4. Qual o total de avaliações feitas?


In [140]:
df['Votes'].sum()

4639654

#### 5. Qual o total de tipos de culinária registrados?


In [141]:
df['Cuisines'].value_counts().count()

166

## Desafio - Pais

#### 1. Qual o nome do país que possui mais cidades registradas?


In [142]:
# Definição das colunas para pesquisa
colunas = ['Country Code', 'City']

contagem_por_pais = df.loc[:, colunas].groupby(['Country Code']).count().reset_index()

# Ordenar pelo maior valor (em ordem decrescente)
contagem_por_pais_sorted = contagem_por_pais.sort_values(by='City', ascending=False).head(10)

contagem_por_pais_sorted

,Country Code,City
4,India,3507
14,United States of America,1395
3,England,437
10,South Africa,382
13,United Arab Emirates,334
1,Brazil,261
6,New Zeland,253
0,Australia,180
2,Canada,180
12,Turkey,165


#### 2. Qual o nome do país que possui mais restaurantes registrados?


In [143]:
# Definição das colunas para pesquisa
colunas = ['Country Code', 'Restaurant ID']

df.loc[:, colunas].groupby(['Country Code']).count().reset_index().sort_values(by='Restaurant ID', ascending=False).head(1)

,Country Code,Restaurant ID
4,India,3507


#### 3. Qual o nome do país que possui mais restaurantes com o nível de preço igual a 4
registrados?

In [144]:
# Definição das colunas para pesquisa
colunas = ['Country Code', 'Aggregate rating']

df.loc[:, colunas].groupby(['Country Code']).mean().reset_index().sort_values(by='Aggregate rating', ascending=False).head(1)

,Country Code,Aggregate rating
5,Indonesia,4.6


#### 4. Qual o nome do país que possui a maior quantidade de tipos de culinária distintos?


In [145]:
# Definição das colunas para pesquisa
colunas = ['Country Code', 'Cuisines']

df.loc[:, colunas].groupby(['Country Code']).nunique().reset_index().sort_values(by='Cuisines', ascending=False).head(1)

,Country Code,Cuisines
4,India,78


#### 5. Qual o nome do país que possui a maior quantidade de avaliações feitas?


In [146]:
# Definição das colunas para pesquisa
colunas = ['Country Code', 'Votes']

df.loc[:, colunas].groupby(['Country Code']).sum().reset_index().sort_values(by='Votes', ascending=False).head(1)

,Country Code,Votes
4,India,3175104


#### 6. Qual o nome do país que possui a maior quantidade de restaurantes que fazem entrega?


In [147]:
# Definição das colunas para pesquisa
colunas = ['Country Code', 'Has Online delivery']

df.loc[:, colunas].groupby(['Country Code']).count().reset_index().sort_values(by='Has Online delivery', ascending=False).head(1)

,Country Code,Has Online delivery
4,India,3507


#### 7. Qual o nome do país que possui a maior quantidade de restaurantes que aceitam reservas?

In [148]:
colunas = ['Country Code', 'Has Table booking']

df.loc[:, colunas].groupby(['Country Code']).count().reset_index().sort_values(by='Has Table booking', ascending=False).head(1)

,Country Code,Has Table booking
4,India,3507


#### 8. Qual o nome do país que possui, na média, a maior quantidade de avaliações registrada?


In [149]:
colunas = ['Country Code', 'Votes']

df.loc[:, colunas].groupby(['Country Code']).mean().reset_index().sort_values(by='Votes', ascending=False).head(1)

,Country Code,Votes
5,Indonesia,1115.317073


#### 9. Qual o nome do país que possui, na média, a maior nota média registrada?

In [150]:
colunas = ['Country Code', 'Aggregate rating']

df.loc[:, colunas].groupby(['Country Code']).mean().reset_index().sort_values(by='Aggregate rating', ascending=False).head(1)

,Country Code,Aggregate rating
5,Indonesia,4.6


#### 10. Qual o nome do país que possui, na média, a menor nota média registrada?



In [151]:
colunas = ['Country Code', 'Aggregate rating']

df.loc[:, colunas].groupby(['Country Code']).mean().reset_index().sort_values(by='Aggregate rating', ascending=True).head(1)

,Country Code,Aggregate rating
1,Brazil,3.349425


#### 11. Qual a média de preço de um prato para dois por país?


In [152]:
colunas = ['Country Code', 'Average Cost for two']

df.loc[:, colunas].groupby(['Country Code']).mean().reset_index().sort_values(by='Average Cost for two', ascending=False)

,Country Code,Average Cost for two
5,Indonesia,312682.926829
0,Australia,138959.783333
11,Sri Lanka,2607.471264
7,Philippines,1211.090909
4,India,702.303964
10,South Africa,335.479058
8,Qatar,176.648936
13,United Arab Emirates,158.218563
9,Singapure,141.036585
1,Brazil,137.375479


## Desafio - Cidade

#### 1. Qual o nome da cidade que possui mais restaurantes registrados?


In [153]:
colunas = ['City', 'Restaurant ID']

df.loc[:, colunas].groupby(['City']).count().reset_index().sort_values(by='Restaurant ID', ascending=False).head(1)




,City,Restaurant ID
76,Nagpur,99


#### 2. Qual o nome da cidade que possui mais restaurantes com nota média acima de 4?


In [154]:
colunas = ['City', 'Aggregate rating']

df.loc[:, colunas].groupby(['City']).mean().reset_index().sort_values(by='Aggregate rating', ascending=False).head(1)

,City,Aggregate rating
74,Muntinlupa City,4.9


#### 3. Qual o nome da cidade que possui mais restaurantes com nota média abaixo de 2.5?

In [155]:
colunas = ['City', 'Restaurant ID']
df.aux = df[df['Aggregate rating'] < 2.5]
df.aux.loc[:, colunas].groupby(['City']).count().reset_index().sort_values(by='Restaurant ID', ascending=False).head(10)

#df.loc[:, colunas].groupby(['City']).count().reset_index().sort_values(by='Restaurant ID', ascending=False)

,City,Restaurant ID
6,Gangtok,47
10,Ooty,22
16,São Paulo,17
1,Brasília,15
14,Rio de Janeiro,13
8,Manchester,7
2,Clarens,4
15,Shimla,4
13,Puducherry,3
5,Edinburgh,2


#### 4. Qual o nome da cidade que possui o maior valor médio de um prato para dois?


In [156]:
colunas = ['City', 'Average Cost for two']

df.loc[:, colunas].groupby(['City']).mean().reset_index().sort_values(by='Average Cost for two', ascending=False).head(1)

,City,Average Cost for two
1,Adelaide,416734.133333


#### 5. Qual o nome da cidade que possui a maior quantidade de tipos de culinária distintas?

In [157]:
colunas = ['City', 'Cuisines']

df.loc[:, colunas].groupby(['City']).count().reset_index().sort_values(by='Cuisines', ascending=False).head(1)

,City,Cuisines
76,Nagpur,99


#### 6. Qual o nome da cidade que possui a maior quantidade de restaurantes que fazem reservas?


In [158]:
colunas = ['City', 'Has Table booking']

df.loc[:, colunas].groupby(['City']).count().reset_index().sort_values(by='Has Table booking', ascending=False).head(1)

,City,Has Table booking
76,Nagpur,99


#### 7. Qual o nome da cidade que possui a maior quantidade de restaurantes que fazem entregas?


In [159]:
colunas = ['City', 'Is delivering now']

df.loc[:, colunas].groupby(['City']).count().reset_index().sort_values(by='Is delivering now', ascending=False).head(1)

,City,Is delivering now
76,Nagpur,99


#### 8. Qual o nome da cidade que possui a maior quantidade de restaurantes que aceitam pedidos online?


In [160]:
colunas = ['City', 'Has Online delivery']

df.loc[:, colunas].groupby(['City']).count().reset_index().sort_values(by='Has Online delivery', ascending=False).head(1)

,City,Has Online delivery
76,Nagpur,99


## Desafio - Restaurantes

#### 1. Qual o nome do restaurante que possui a maior quantidade de avaliações?


In [161]:
colunas = ['Restaurant Name', 'Votes']

df.loc[:, colunas].groupby(['Restaurant Name']).count().reset_index().sort_values(by='Votes', ascending=False).head(1)

,Restaurant Name,Votes
1528,Domino's Pizza,204


#### 2. Qual o nome do restaurante com a maior nota média?


In [162]:
colunas = ['Restaurant Name', 'Aggregate rating']

df.loc[:, colunas].groupby(['Restaurant Name']).mean().reset_index().sort_values(by='Aggregate rating', ascending=False).head(1)

,Restaurant Name,Aggregate rating
1580,Earl of Sandwich,4.9


#### 3. Qual o nome do restaurante que possui o maior valor de uma prato para duas pessoas?

In [163]:
colunas = ['Restaurant Name', 'Average Cost for two']

df.loc[:, colunas].sort_values(by='Average Cost for two', ascending=False).head(1)

,Restaurant Name,Average Cost for two
385,d'Arry's Verandah Restaurant,25000017


#### 4. Qual o nome do restaurante de tipo de culinária brasileira que possui a menor média de avaliação?

In [164]:
colunas = ['Restaurant Name', 'Aggregate rating']

# Filtro para cozinhas Brasileiras
df_aux = df[df['Cuisines'] == 'Brazilian']

df_aux.loc[:, colunas].groupby(['Restaurant Name']).mean().reset_index().sort_values(by='Aggregate rating', ascending=True).head(1)

,Restaurant Name,Aggregate rating
54,Restaurante dos Amigos,0.0


#### 5. Qual o nome do restaurante de tipo de culinária brasileira, e que é do Brasil, que possui a maior média de avaliação?

In [169]:
colunas = ['Restaurant Name', 'Aggregate rating']

# Filtro para cozinhas Brasileiras
df_aux = df[df['Cuisines'] == 'Brazilian']

df_aux.loc[:, 'Restaurant Name'].groupby(['Restaurant Name']).mean().reset_index().sort_values(by='Aggregate rating', ascending=False).head(1)


KeyError: 'Cuisines'

In [168]:
# Exemplo de DataFrame
data = {'Nome': ['Alice', 'Bob', 'Carol'],
        'Idade': [25, 30, 27]}
df = pd.DataFrame(data)
df
# Extrair o valor da célula na linha 1, coluna 'Nome' e converter para string
valor = df.at[1, 'Nome']
valor_str = str(valor)
print(valor_str)  # Saída: 'Bob'

Bob


#### 6. Os restaurantes que aceitam pedido online são também, na média, os restaurantes que mais possuem avaliações registradas?

In [167]:
colunas = ['Restaurant Name', 'Has Online delivery', 'Votes']

print("Não tem relação, veja no dataframe abaixo que a quantidade de votos não tem relação com entregas online:")
df.loc[:, colunas].sort_values(by='Votes', ascending=False).head(10)


Não tem relação, veja no dataframe abaixo que a quantidade de votos não tem relação com entregas online:


KeyError: "None of [Index(['Restaurant Name', 'Has Online delivery', 'Votes'], dtype='object')] are in the [columns]"

#### 7. Os restaurantes que fazem reservas são também, na média, os restaurantes que possuem o maior valor médio de um prato para duas pessoas?

In [ ]:
# Primeiro filtrar restaurantes com preço médio para dias pessoas
colunas2 = ['Restaurant Name', 'Average Cost for two', 'Has Table booking']
print("Não, veja no dataframe que as duas informações não tem relação:")
df.loc[:, colunas2].groupby(['Restaurant Name']).mean().reset_index().sort_values(by='Average Cost for two', ascending=False).head(10)

#### 8. Os restaurantes do tipo de culinária japonesa dos Estados Unidos da América possuem um valor médio de prato para duas pessoas maior que as churrascarias americanas (BBQ)?


In [ ]:
# Primeiro filtrar apenas restaurantes dos EUA
df_aux = df[df['Country Code'] == 'United States of America']

df_aux2 = df_aux[(df_aux['Cuisines'] == 'Japanese') | (df_aux['Cuisines'] == 'BBQ')]

colunas = ['Cuisines', 'Average Cost for two']

print("Sim, conforme dataframe abaixo a culinária japonesa é mais caro em média:")
df_aux2.loc[:, colunas].groupby(['Cuisines']).mean().reset_index().sort_values(by='Average Cost for two', ascending=False).head(10)

## Desafio - Culinária

#### 1. Dos restaurantes que possuem o tipo de culinária italiana, qual o nome do restaurante com a maior média de avaliação?

In [ ]:
colunas = ['Restaurant Name', 'Aggregate rating']

df_aux = df[df['Cuisines'] == 'Italian']

df_aux.loc[:, colunas].groupby(['Restaurant Name']).mean().reset_index().sort_values(by='Aggregate rating', ascending=False).head(1)


#### 2. Dos restaurantes que possuem o tipo de culinária italiana, qual o nome do restaurante com a menor média de avaliação?

In [ ]:
colunas = ['Restaurant Name', 'Aggregate rating']

df_aux = df[df['Cuisines'] == 'Italian']

df_aux.loc[:, colunas].groupby(['Restaurant Name']).mean().reset_index().sort_values(by='Aggregate rating', ascending=True).head(1)


#### 3. Dos restaurantes que possuem o tipo de culinária americana, qual o nome do restaurante com a maior média de avaliação?


In [ ]:
colunas = ['Restaurant Name', 'Aggregate rating']

df_aux = df[df['Cuisines'] == 'American']

df_aux.loc[:, colunas].groupby(['Restaurant Name']).mean().reset_index().sort_values(by='Aggregate rating', ascending=False).head(1)


#### 4. Dos restaurantes que possuem o tipo de culinária americana, qual o nome do restaurante com a menor média de avaliação?

In [ ]:
colunas = ['Restaurant Name', 'Aggregate rating']

df_aux = df[df['Cuisines'] == 'American']

df_aux.loc[:, colunas].groupby(['Restaurant Name']).mean().reset_index().sort_values(by='Aggregate rating', ascending=True).head(1)


#### 5. Dos restaurantes que possuem o tipo de culinária árabe, qual o nome do restaurante com a maior média de avaliação?

In [ ]:
colunas = ['Restaurant Name', 'Aggregate rating']

df_aux = df[df['Cuisines'] == 'Arabian']

df_aux.loc[:, colunas].groupby(['Restaurant Name']).mean().reset_index().sort_values(by='Aggregate rating', ascending=False).head(1)


#### 6. Dos restaurantes que possuem o tipo de culinária árabe, qual o nome do restaurante com a menor média de avaliação?

In [ ]:
colunas = ['Restaurant Name', 'Aggregate rating']

df_aux = df[df['Cuisines'] == 'Arabian']

df_aux.loc[:, colunas].groupby(['Restaurant Name']).mean().reset_index().sort_values(by='Aggregate rating', ascending=True).head(1)


#### 7. Dos restaurantes que possuem o tipo de culinária japonesa, qual o nome do restaurante com a maior média de avaliação?


In [ ]:
colunas = ['Restaurant Name', 'Aggregate rating']

df_aux = df[df['Cuisines'] == 'Japanese']

df_aux.loc[:, colunas].groupby(['Restaurant Name']).mean().reset_index().sort_values(by='Aggregate rating', ascending=False).head(1)


#### 8. Dos restaurantes que possuem o tipo de culinária japonesa, qual o nome do restaurante com a menor média de avaliação?

In [ ]:
colunas = ['Restaurant Name', 'Aggregate rating']

df_aux = df[df['Cuisines'] == 'Japanese']

df_aux.loc[:, colunas].groupby(['Restaurant Name']).mean().reset_index().sort_values(by='Aggregate rating', ascending=True).head(1)


#### 9. Dos restaurantes que possuem o tipo de culinária caseira, qual o nome do restaurante com a maior média de avaliação?

In [ ]:
colunas = ['Restaurant Name', 'Aggregate rating']

df_aux = df[df['Cuisines'] == 'Home-made']

df_aux.loc[:, colunas].groupby(['Restaurant Name']).mean().reset_index().sort_values(by='Aggregate rating', ascending=False).head(1)


#### 10. Dos restaurantes que possuem o tipo de culinária caseira, qual o nome do restaurante com a menor média de avaliação?


In [ ]:
colunas = ['Restaurant Name', 'Aggregate rating']

df_aux = df[df['Cuisines'] == 'Home-made']

df_aux.loc[:, colunas].groupby(['Restaurant Name']).mean().reset_index().sort_values(by='Aggregate rating', ascending=True).head(1)


#### 11. Qual o tipo de culinária que possui o maior valor médio de um prato para duas pessoas?

In [ ]:
colunas = ['Cuisines', 'Average Cost for two']

df.loc[:, colunas].groupby(['Cuisines']).mean().reset_index().sort_values(by='Average Cost for two', ascending=False).head(1)

#### 12. Qual o tipo de culinária que possui a maior nota média?


In [ ]:
colunas = ['Cuisines', 'Aggregate rating']

df.loc[:, colunas].groupby(['Cuisines']).mean().reset_index().sort_values(by='Aggregate rating', ascending=False).head(1)

#### 13. Qual o tipo de culinária que possui mais restaurantes que aceitam pedidos online e fazem entregas?


In [ ]:
df_aux = df[(df['Has Online delivery'] == True) & (df['Is delivering now'] == True)]

colunas = ['Cuisines', 'Restaurant ID']
df_aux.loc[:, colunas].groupby(['Cuisines']).count().reset_index().sort_values(by='Restaurant ID', ascending=False).head(1)



In [173]:
df_aux['Cuisines'].values

array(['Brazilian', 'Brazilian', 'Brazilian', 'Brazilian', 'Brazilian',
       'Brazilian', 'Brazilian', 'Brazilian', 'Brazilian', 'Brazilian',
       'Brazilian', 'Brazilian', 'Brazilian', 'Brazilian', 'Brazilian',
       'Brazilian', 'Brazilian', 'Brazilian', 'Brazilian', 'Brazilian',
       'Brazilian', 'Brazilian', 'Brazilian', 'Brazilian', 'Brazilian',
       'Brazilian', 'Brazilian', 'Brazilian', 'Brazilian', 'Brazilian',
       'Brazilian', 'Brazilian', 'Brazilian', 'Brazilian', 'Brazilian',
       'Brazilian', 'Brazilian', 'Brazilian', 'Brazilian', 'Brazilian',
       'Brazilian', 'Brazilian', 'Brazilian', 'Brazilian', 'Brazilian',
       'Brazilian', 'Brazilian', 'Brazilian', 'Brazilian', 'Brazilian',
       'Brazilian', 'Brazilian', 'Brazilian', 'Brazilian', 'Brazilian',
       'Brazilian', 'Brazilian', 'Brazilian', 'Brazilian', 'Brazilian',
       'Brazilian', 'Brazilian', 'Brazilian', 'Brazilian', 'Brazilian',
       'Brazilian', 'Brazilian', 'Brazilian', 'Brazilian', 'Braz

In [ ]:
df_aux.head()